In [72]:
import numpy as np
from cvxopt import solvers, matrix, spmatrix, spdiag, sparse
import matplotlib.pyplot as plt
data = np.load('ex2q2_mnist.npz', allow_pickle=True)

In [73]:
def softsvm(l, trainX: np.array, trainy: np.array):
    """
    :param l: the parameter lambda of the soft SVM algorithm
    :param trainX: numpy array of size (m, d) containing the training sample
    :param trainy: numpy array of size (m, 1) containing the labels of the training sample
    :return: linear predictor w, a numpy array of size (d, 1)
    """
    m, d = trainX.shape
    u = np.concatenate((np.full(d, 0), np.full(m, 1/m)))
    H = np.zeros((d + m, d + m))
    for i in range(d):
        H[i][i] = 2 * l
        
    A_top_left = np.zeros((m, d))
    A_top_right = np.identity(m)
    A_bottom_left = trainX * trainy.reshape(-1, 1)
    A_bottom_right = np.identity(m)
    A = np.vstack([np.hstack([A_top_left, A_top_right]),
                   np.hstack([A_bottom_left, A_bottom_right])])
    
    v = np.hstack((np.zeros(m), np.ones(m)))
    sol = solvers.qp(matrix(H), matrix(u), -matrix(A), -matrix(v))
    w = np.array(sol["x"])[:d]
    return w


In [74]:
def predict_svm(w: np.array, x_test: np.array):
    predictions = np.array([[np.sign(test @ w) for test in x_test]])
    return predictions

def simple_test():
    # load question 2 data
    # data = np.load('ex2q2_mnist.npz')
    trainX = data['Xtrain']
    testX = data['Xtest']
    trainy = data['Ytrain']
    testy = data['Ytest']

    m = 100
    d = trainX.shape[1]

    # Get a random m training examples from the training set
    indices = np.random.permutation(trainX.shape[0])
    _trainX = trainX[indices[:m]]
    _trainy = trainy[indices[:m]]

    # run the softsvm algorithm
    w = softsvm(10, _trainX, _trainy)

    # tests to make sure the output is of the intended class and shape
    assert isinstance(w, np.ndarray), "The output of the function softsvm should be a numpy array"
    assert w.shape[0] == d and w.shape[1] == 1, f"The shape of the output should be ({d}, 1)"

    # get a random example from the test set, and classify it
    i = np.random.randint(0, testX.shape[0])
    predicty = np.sign(testX[i] @ w)

    # this line should print the classification of the i'th test sample (1 or -1).
    print(f"The {i}'th test sample was classified as {predicty}")

In [75]:
simple_test()

     pcost       dcost       gap    pres   dres
 0: -9.0413e-03  1.0010e+00  2e+02  1e+00  8e+04
 1:  9.7142e-01 -1.9790e+00  3e+00  1e-02  8e+02
 2:  2.6678e-01 -2.2585e-02  3e-01  3e-04  1e+01
 3:  6.2202e-02 -4.1471e-03  7e-02  5e-05  3e+00
 4:  1.5012e-02 -4.2793e-04  2e-02  1e-05  6e-01
 5:  1.7828e-03  5.8769e-04  1e-03  7e-07  4e-02
 6:  7.5936e-04  6.7308e-04  9e-05  4e-08  2e-03
 7:  6.9057e-04  6.8448e-04  6e-06  2e-09  1e-04
 8:  6.8608e-04  6.8591e-04  2e-07  3e-11  1e-06
 9:  6.8597e-04  6.8597e-04  4e-09  4e-13  2e-08
Optimal solution found.
The 969'th test sample was classified as [-1.]
